In [1]:
# === IMPORTACIONES NECESARIAS ===
import pandas as pd
import numpy as np
import gc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras import backend as K

In [2]:
# === CARGA Y PREPROCESAMIENTO DE DATOS ===
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/redes/indicadores-filtrados-premier-3-cambios.csv'
df = pd.read_csv(path, sep=",")
# Eliminar filas con valores faltantes en la variable objetivo
df = df.dropna(subset=["resultado_local"]).reset_index(drop=True)
df = df.dropna(subset=["resultado_visitante"]).reset_index(drop=True)

# Variable objetivo: número de goles del equipo local
y_local = df["resultado_local"]
y_visitante = df["resultado_visitante"]

# Variables predictoras: eliminamos variables respuesta y otras no informativas
X = df.drop(columns=[
    "resultado_partido", "resultado_local", "resultado_visitante",
    "jornada", "id_indicadores_equipo_prepartido", "id_partido", "temporada"
])

Mounted at /content/drive


In [3]:
# Escalado
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# División en train/test
X_train_local, X_test_local, y_train_local, y_test_local = train_test_split(X_scaled, y_local, test_size=0.2, random_state=42)
X_train_visitante, X_test_visitante, y_train_visitante, y_test_visitante = train_test_split(X_scaled, y_visitante, test_size=0.2, random_state=42)

In [4]:
# === HIPERPARÁMETROS DEL GRID ===
opciones_epocas = [50, 80]
opciones_batch = [16, 48]
opciones_optimizador = ["SGD", "adam"]
opciones_callbacks = [0, 1]
opciones_red = [0, 1, 2, 3, 4, 5, 6]

# Inicializar resultados
resultsDF = pd.DataFrame(columns=["epocas", "batch", "optimizador", "callbacks", "red", "mae"])

# LOCAL

In [5]:
# === GRID SEARCH PARA REGRESIÓN ===
for epocas in opciones_epocas:
    for batch in opciones_batch:
        for opt in opciones_optimizador:
            for cb in opciones_callbacks:
                for red in opciones_red:
                    if red == 0:
                        model = Sequential([
                            Dense(64, activation='relu', input_shape=(X_train_local.shape[1],)),
                            Dense(1)
                        ])

                    elif red == 1:
                        model = Sequential([
                            Dense(128, activation='relu', input_shape=(X_train_local.shape[1],)),
                            Dense(64, activation='relu'),
                            Dense(32, activation='relu'),
                            Dense(1)
                        ])

                    elif red == 2:
                        model = Sequential([
                            Dense(256, activation='relu', input_shape=(X_train_local.shape[1],)),
                            Dense(128, activation='relu'),
                            Dropout(0.4),
                            Dense(64, activation='relu'),
                            Dropout(0.3),
                            Dense(1)
                        ])

                    elif red == 3:
                        model = Sequential([
                            Dense(128, activation='relu', kernel_regularizer=l2(0.001), input_shape=(X_train_local.shape[1],)),
                            BatchNormalization(),
                            Dropout(0.3),
                            Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
                            BatchNormalization(),
                            Dense(32, activation='relu'),
                            Dense(1)
                        ])

                    elif red == 4:
                        model = Sequential([
                            Dense(128, activation='relu', kernel_initializer=HeNormal(), input_shape=(X_train_local.shape[1],)),
                            BatchNormalization(),
                            Dropout(0.3),
                            Dense(64, activation='relu', kernel_initializer=HeNormal()),
                            BatchNormalization(),
                            Dense(32, activation='relu'),
                            Dropout(0.2),
                            Dense(1)
                        ])

                    elif red == 5:
                        model = Sequential([
                            Dense(128, activation='relu', input_shape=(X_train_local.shape[1],)),
                            Dense(64, activation='relu'),
                            Dense(32, activation='relu'),
                            Dense(64, activation='relu'),
                            Dense(1)
                        ])

                    elif red == 6:
                        model = Sequential([
                            Dense(256, activation='relu', kernel_initializer=HeNormal(), input_shape=(X_train_local.shape[1],)),
                            BatchNormalization(),
                            Dropout(0.3),
                            Dense(128, activation='relu'),
                            BatchNormalization(),
                            Dropout(0.3),
                            Dense(32, activation='relu'),
                            Dense(1)
                        ])

                    # Compilar modelo
                    model.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])

                    # Callbacks
                    callbacks_list = []
                    if cb == 1:
                        early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
                        reduce_lr = ReduceLROnPlateau(factor=0.2, patience=5)
                        callbacks_list = [early_stopping, reduce_lr]

                    # Entrenamiento
                    history = model.fit(
                        X_train_local, y_train_local,
                        epochs=epocas,
                        batch_size=batch,
                        validation_split=0.2,
                        callbacks=callbacks_list,
                        verbose=0
                    )

                    # Evaluación
                    loss, mae = model.evaluate(X_test_local, y_test_local, verbose=0)

                    # Guardar el mejor modelo por tipo
                    model_path = f"/content/drive/MyDrive/Colab Notebooks/redes/mejor_modelo_red_{red}_goles_local_premier-3-cambios.h5"
                    try:
                        modelo_guardado = load_model(model_path)
                        _, mae_guardado = modelo_guardado.evaluate(X_test_local, y_test_local, verbose=0)
                        if mae < mae_guardado:
                            model.save(model_path)
                            print(f"✅ Modelo actualizado para red {red} con MAE {mae:.4f}")
                    except:
                        model.save(model_path)
                        print(f"📁 Modelo guardado por primera vez para red {red} con MAE {mae:.4f}")

                    print(f"Red {red} | Opt: {opt} | Ep: {epocas} | Batch: {batch} | CB: {cb} → MAE: {mae:.4f}")

                    # Guardar en DataFrame
                    resultsDF.loc[len(resultsDF)] = [epocas, batch, opt, cb, red, mae]

# === EXPORTAR RESULTADOS ===
resultsDF.to_csv("/content/drive/MyDrive/Colab Notebooks/redes/modelos_goles_local_premier-3-cambios.csv", index=False)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


📁 Modelo guardado por primera vez para red 0 con MAE 1.0845
Red 0 | Opt: SGD | Ep: 50 | Batch: 16 | CB: 0 → MAE: 1.0845


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


📁 Modelo guardado por primera vez para red 1 con MAE 1.0943
Red 1 | Opt: SGD | Ep: 50 | Batch: 16 | CB: 0 → MAE: 1.0943


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

Red 4 | Opt: adam | Ep: 80 | Batch: 48 | CB: 1 → MAE: 1.0759


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Red 5 | Opt: adam | Ep: 80 | Batch: 48 | CB: 1 → MAE: 1.0644


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Red 6 | Opt: adam | Ep: 80 | Batch: 48 | CB: 1 → MAE: 1.0701


In [6]:
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np

# Paso 1: Cargar resultados del grid search
df_resultados = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/redes/modelos_goles_local_premier-3-cambios.csv")

# Inicialización
mejor_mae = float("inf")
mejor_modelo = None
mejor_red = None

print("📊 Evaluación de los mejores modelos por red:\n")

# Evaluar mejor modelo de cada red
for red in range(7):
    try:
        # Cargar modelo guardado
        path = f"/content/drive/MyDrive/Colab Notebooks/redes/mejor_modelo_red_{red}_goles_local_premier-3-cambios.h5"
        model = load_model(path)
        loss, mae = model.evaluate(X_test_local, y_test_local, verbose=0)

        # Buscar mejor configuración en CSV
        config_red = df_resultados[df_resultados["red"] == red].sort_values(by="mae").iloc[0]

        print(f"🔢 Red {red} --> MAE = {mae:.4f}")
        print("   ⚙️ Hiperparámetros:")
        print(f"   - Épocas:       {config_red['epocas']}")
        print(f"   - Batch size:   {config_red['batch']}")
        print(f"   - Optimizador:  {config_red['optimizador']}")
        print(f"   - Callbacks:    {'Sí' if config_red['callbacks'] else 'No'}")
        print()

        # Actualizar mejor modelo global
        if mae < mejor_mae:
            mejor_mae = mae
            mejor_modelo = model
            mejor_red = red

    except Exception as e:
        print(f"❌ Error con red {red}: {e}\n")

# Mostrar resumen del mejor modelo
print(f"🏆 Mejor modelo global: red {mejor_red} con MAE = {mejor_mae:.4f}")


# Paso 2: Obtener pesos del primer Dense layer
primer_dense = None
for layer in mejor_modelo.layers:
    if "Dense" in layer.__class__.__name__:
        primer_dense = layer
        break

if primer_dense is not None:
    pesos, _ = primer_dense.get_weights()  # pesos.shape = (n_variables, n_neuronas)
    importancia = np.mean(np.abs(pesos), axis=1)  # media de pesos por variable (input)

    # Paso 3: Asociar importancia con nombres de columnas
    nombres_variables = X.columns  # asegúrate de que X esté sin escalar
    importancia_df = pd.DataFrame({
        "variable": nombres_variables,
        "importancia": importancia
    })

    top_15 = importancia_df.sort_values(by="importancia", ascending=False).head(15)
    print("\n🔝 Top 15 variables más importantes según los pesos del primer layer:\n")
    print(top_15.to_string(index=False))
else:
    print("❌ No se encontró una capa Dense inicial en el mejor modelo.")


📊 Evaluación de los mejores modelos por red:



🔢 Red 0 --> MAE = 1.0160
   ⚙️ Hiperparámetros:
   - Épocas:       80
   - Batch size:   16
   - Optimizador:  adam
   - Callbacks:    Sí



🔢 Red 1 --> MAE = 1.0388
   ⚙️ Hiperparámetros:
   - Épocas:       80
   - Batch size:   16
   - Optimizador:  SGD
   - Callbacks:    Sí



🔢 Red 2 --> MAE = 1.0308
   ⚙️ Hiperparámetros:
   - Épocas:       80
   - Batch size:   16
   - Optimizador:  adam
   - Callbacks:    Sí



🔢 Red 3 --> MAE = 1.0288
   ⚙️ Hiperparámetros:
   - Épocas:       50
   - Batch size:   48
   - Optimizador:  adam
   - Callbacks:    No



🔢 Red 4 --> MAE = 1.0334
   ⚙️ Hiperparámetros:
   - Épocas:       80
   - Batch size:   48
   - Optimizador:  SGD
   - Callbacks:    Sí



🔢 Red 5 --> MAE = 1.0288
   ⚙️ Hiperparámetros:
   - Épocas:       50
   - Batch size:   16
   - Optimizador:  adam
   - Callbacks:    Sí

🔢 Red 6 --> MAE = 1.0271
   ⚙️ Hiperparámetros:
   - Épocas:       50
   - Batch size:   48
   - Optimizador:  adam
   - Callbacks:    Sí

🏆 Mejor modelo global: red 0 con MAE = 1.0160

🔝 Top 15 variables más importantes según los pesos del primer layer:

                                                          variable  importancia
                  proporcion visitante cambios goleadores en sitio     0.095966
              proporcion visitante cambios antes descanso en sitio     0.093088
                         proporcion local cambios asistentes sitio     0.092692
                                   proporcion local rojas en sitio     0.091636
                   porcentaje visitante mas 0,5 encajados en sitio     0.090679
                             media visitante cambios minutos sitio     0.090642
                                 media local 

In [7]:
# Paso 4: Evaluar peso total de variables que contienen "cambios"
importancia_total = importancia_df["importancia"].sum()

# Filtrar variables que contienen "cambios" (case insensitive)
importancia_cambios = importancia_df[importancia_df["variable"].str.contains("cambios", case=False)]

suma_cambios = importancia_cambios["importancia"].sum()
proporcion = (suma_cambios / importancia_total) * 100

print(f"\n📈 Proporción de importancia atribuida a variables relacionadas con 'cambios': {proporcion:.2f}% del total")



📈 Proporción de importancia atribuida a variables relacionadas con 'cambios': 45.59% del total


In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Add

input_layer = Input(shape=(X_train_local.shape[1],))
x = Dense(128, activation='relu')(input_layer)
x_skip = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x_skip)
x = Add()([x, x_skip])
output = Dense(1)(x)

modelo_residual = Model(inputs=input_layer, outputs=output)
modelo_residual.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Entrenamiento
modelo_residual.fit(X_train_local, y_train_local, epochs=80, batch_size=32, validation_split=0.2, verbose=0)

# Evaluación
loss, mae = modelo_residual.evaluate(X_test_local, y_test_local, verbose=0)
print(f"🔁 Red residual → MAE: {mae:.4f}")


🔁 Red residual → MAE: 1.2567


In [9]:
from tensorflow.keras.layers import Multiply, Softmax

input_layer = Input(shape=(X_train_local.shape[1],))
attention = Dense(X_train_local.shape[1], activation='softmax')(input_layer)
x = Multiply()([input_layer, attention])  # aplica atención

x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
output = Dense(1)(x)

modelo_atencion = Model(inputs=input_layer, outputs=output)
modelo_atencion.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Entrenamiento
modelo_atencion.fit(X_train_local, y_train_local, epochs=80, batch_size=32, validation_split=0.2, verbose=0)

# Evaluación
loss, mae = modelo_atencion.evaluate(X_test_local, y_test_local, verbose=0)
print(f"🎯 Red con atención → MAE: {mae:.4f}")


🎯 Red con atención → MAE: 1.1414


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Dropout

modelo_embudo = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_local.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1)
])
modelo_embudo.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Entrenamiento
modelo_embudo.fit(X_train_local, y_train_local, epochs=80, batch_size=32, validation_split=0.2, verbose=0)

# Evaluación
loss, mae = modelo_embudo.evaluate(X_test_local, y_test_local, verbose=0)
print(f"🏗️ Red tipo embudo → MAE: {mae:.4f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


🏗️ Red tipo embudo → MAE: 1.1728


# VISITANTE

In [11]:
# === GRID SEARCH PARA REGRESIÓN ===
for epocas in opciones_epocas:
    for batch in opciones_batch:
        for opt in opciones_optimizador:
            for cb in opciones_callbacks:
                for red in opciones_red:
                    if red == 0:
                        model = Sequential([
                            Dense(64, activation='relu', input_shape=(X_train_visitante.shape[1],)),
                            Dense(1)
                        ])

                    elif red == 1:
                        model = Sequential([
                            Dense(128, activation='relu', input_shape=(X_train_visitante.shape[1],)),
                            Dense(64, activation='relu'),
                            Dense(32, activation='relu'),
                            Dense(1)
                        ])

                    elif red == 2:
                        model = Sequential([
                            Dense(256, activation='relu', input_shape=(X_train_visitante.shape[1],)),
                            Dense(128, activation='relu'),
                            Dropout(0.4),
                            Dense(64, activation='relu'),
                            Dropout(0.3),
                            Dense(1)
                        ])

                    elif red == 3:
                        model = Sequential([
                            Dense(128, activation='relu', kernel_regularizer=l2(0.001), input_shape=(X_train_visitante.shape[1],)),
                            BatchNormalization(),
                            Dropout(0.3),
                            Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
                            BatchNormalization(),
                            Dense(32, activation='relu'),
                            Dense(1)
                        ])

                    elif red == 4:
                        model = Sequential([
                            Dense(128, activation='relu', kernel_initializer=HeNormal(), input_shape=(X_train_visitante.shape[1],)),
                            BatchNormalization(),
                            Dropout(0.3),
                            Dense(64, activation='relu', kernel_initializer=HeNormal()),
                            BatchNormalization(),
                            Dense(32, activation='relu'),
                            Dropout(0.2),
                            Dense(1)
                        ])

                    elif red == 5:
                        model = Sequential([
                            Dense(128, activation='relu', input_shape=(X_train_visitante.shape[1],)),
                            Dense(64, activation='relu'),
                            Dense(32, activation='relu'),
                            Dense(64, activation='relu'),
                            Dense(1)
                        ])

                    elif red == 6:
                        model = Sequential([
                            Dense(256, activation='relu', kernel_initializer=HeNormal(), input_shape=(X_train_visitante.shape[1],)),
                            BatchNormalization(),
                            Dropout(0.3),
                            Dense(128, activation='relu'),
                            BatchNormalization(),
                            Dropout(0.3),
                            Dense(32, activation='relu'),
                            Dense(1)
                        ])

                    # Compilar modelo
                    model.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])

                    # Callbacks
                    callbacks_list = []
                    if cb == 1:
                        early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
                        reduce_lr = ReduceLROnPlateau(factor=0.2, patience=5)
                        callbacks_list = [early_stopping, reduce_lr]

                    # Entrenamiento
                    history = model.fit(
                        X_train_visitante, y_train_visitante,
                        epochs=epocas,
                        batch_size=batch,
                        validation_split=0.2,
                        callbacks=callbacks_list,
                        verbose=0
                    )

                    # Evaluación
                    loss, mae = model.evaluate(X_test_visitante, y_test_visitante, verbose=0)

                    # Guardar el mejor modelo por tipo
                    model_path = f"/content/drive/MyDrive/Colab Notebooks/redes/mejor_modelo_red_{red}_goles_visitante_premier-3-cambios.h5"
                    try:
                        modelo_guardado = load_model(model_path)
                        _, mae_guardado = modelo_guardado.evaluate(X_test_visitante, y_test_visitante, verbose=0)
                        if mae < mae_guardado:
                            model.save(model_path)
                            print(f"✅ Modelo actualizado para red {red} con MAE {mae:.4f}")
                    except:
                        model.save(model_path)
                        print(f"📁 Modelo guardado por primera vez para red {red} con MAE {mae:.4f}")

                    print(f"Red {red} | Opt: {opt} | Ep: {epocas} | Batch: {batch} | CB: {cb} → MAE: {mae:.4f}")

                    # Guardar en DataFrame
                    resultsDF.loc[len(resultsDF)] = [epocas, batch, opt, cb, red, mae]

# === EXPORTAR RESULTADOS ===
resultsDF.to_csv("/content/drive/MyDrive/Colab Notebooks/redes/modelos_goles_visitante_premier-3-cambios.csv", index=False)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


📁 Modelo guardado por primera vez para red 0 con MAE 1.0045
Red 0 | Opt: SGD | Ep: 50 | Batch: 16 | CB: 0 → MAE: 1.0045


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


📁 Modelo guardado por primera vez para red 1 con MAE 1.0225
Red 1 | Opt: SGD | Ep: 50 | Batch: 16 | CB: 0 → MAE: 1.0225


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

Red 4 | Opt: adam | Ep: 80 | Batch: 48 | CB: 1 → MAE: 0.8688


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Red 5 | Opt: adam | Ep: 80 | Batch: 48 | CB: 1 → MAE: 0.8982


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Red 6 | Opt: adam | Ep: 80 | Batch: 48 | CB: 1 → MAE: 0.9514


In [12]:
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np

# Paso 1: Cargar resultados del grid search
df_resultados = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/redes/modelos_goles_visitante_premier-3-cambios.csv")

# Inicialización
mejor_mae = float("inf")
mejor_modelo = None
mejor_red = None

print("📊 Evaluación de los mejores modelos por red:\n")

# Evaluar mejor modelo de cada red
for red in range(7):
    try:
        # Cargar modelo guardado
        path = f"/content/drive/MyDrive/Colab Notebooks/redes/mejor_modelo_red_{red}_goles_visitante_premier-3-cambios.h5"
        model = load_model(path)
        loss, mae = model.evaluate(X_test_visitante, y_test_visitante, verbose=0)

        # Buscar mejor configuración en CSV
        config_red = df_resultados[df_resultados["red"] == red].sort_values(by="mae").iloc[0]

        print(f"🔢 Red {red} --> MAE = {mae:.4f}")
        print("   ⚙️ Hiperparámetros:")
        print(f"   - Épocas:       {config_red['epocas']}")
        print(f"   - Batch size:   {config_red['batch']}")
        print(f"   - Optimizador:  {config_red['optimizador']}")
        print(f"   - Callbacks:    {'Sí' if config_red['callbacks'] else 'No'}")
        print()

        # Actualizar mejor modelo global
        if mae < mejor_mae:
            mejor_mae = mae
            mejor_modelo = model
            mejor_red = red

    except Exception as e:
        print(f"❌ Error con red {red}: {e}\n")

# Mostrar resumen del mejor modelo
print(f"🏆 Mejor modelo global: red {mejor_red} con MAE = {mejor_mae:.4f}")


# Paso 2: Obtener pesos del primer Dense layer
primer_dense = None
for layer in mejor_modelo.layers:
    if "Dense" in layer.__class__.__name__:
        primer_dense = layer
        break

if primer_dense is not None:
    pesos, _ = primer_dense.get_weights()  # pesos.shape = (n_variables, n_neuronas)
    importancia = np.mean(np.abs(pesos), axis=1)  # media de pesos por variable (input)

    # Paso 3: Asociar importancia con nombres de columnas
    nombres_variables = X.columns  # asegúrate de que X esté sin escalar
    importancia_df = pd.DataFrame({
        "variable": nombres_variables,
        "importancia": importancia
    })

    top_15 = importancia_df.sort_values(by="importancia", ascending=False).head(15)
    print("\n🔝 Top 15 variables más importantes según los pesos del primer layer:\n")
    print(top_15.to_string(index=False))
else:
    print("❌ No se encontró una capa Dense inicial en el mejor modelo.")


📊 Evaluación de los mejores modelos por red:



🔢 Red 0 --> MAE = 0.8639
   ⚙️ Hiperparámetros:
   - Épocas:       80
   - Batch size:   48
   - Optimizador:  SGD
   - Callbacks:    Sí



🔢 Red 1 --> MAE = 0.8617
   ⚙️ Hiperparámetros:
   - Épocas:       50
   - Batch size:   16
   - Optimizador:  SGD
   - Callbacks:    Sí



🔢 Red 2 --> MAE = 0.8259
   ⚙️ Hiperparámetros:
   - Épocas:       50
   - Batch size:   48
   - Optimizador:  SGD
   - Callbacks:    No



🔢 Red 3 --> MAE = 0.8209
   ⚙️ Hiperparámetros:
   - Épocas:       50
   - Batch size:   48
   - Optimizador:  adam
   - Callbacks:    Sí



🔢 Red 4 --> MAE = 0.8192
   ⚙️ Hiperparámetros:
   - Épocas:       50
   - Batch size:   16
   - Optimizador:  adam
   - Callbacks:    Sí



🔢 Red 5 --> MAE = 0.8553
   ⚙️ Hiperparámetros:
   - Épocas:       50
   - Batch size:   48
   - Optimizador:  SGD
   - Callbacks:    No

🔢 Red 6 --> MAE = 0.8455
   ⚙️ Hiperparámetros:
   - Épocas:       50
   - Batch size:   48
   - Optimizador:  SGD
   - Callbacks:    No

🏆 Mejor modelo global: red 4 con MAE = 0.8192

🔝 Top 15 variables más importantes según los pesos del primer layer:

                                                        variable  importancia
                  porcentaje visitante mas 0,5 marcados en sitio     0.101530
proporcion local cambios centrocampistas a delanteros en general     0.100710
                        proporcion visitante posesion en general     0.100574
                   proporcion visitante goles encajados en sitio     0.098611
 proporcion visitante cambios alineacion centrocampista en sitio     0.098487
       proporcion local cambios defensas a centrocampistas sitio     0.098023
                     proporcion local corners a favor en gene

In [13]:
# Paso 4: Evaluar peso total de variables que contienen "cambios"
importancia_total = importancia_df["importancia"].sum()

# Filtrar variables que contienen "cambios" (case insensitive)
importancia_cambios = importancia_df[importancia_df["variable"].str.contains("cambios", case=False)]

suma_cambios = importancia_cambios["importancia"].sum()
proporcion = (suma_cambios / importancia_total) * 100

print(f"\n📈 Proporción de importancia atribuida a variables relacionadas con 'cambios': {proporcion:.2f}% del total")



📈 Proporción de importancia atribuida a variables relacionadas con 'cambios': 45.16% del total


In [14]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Add

input_layer = Input(shape=(X_train_visitante.shape[1],))
x = Dense(128, activation='relu')(input_layer)
x_skip = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x_skip)
x = Add()([x, x_skip])
output = Dense(1)(x)

modelo_residual = Model(inputs=input_layer, outputs=output)
modelo_residual.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Entrenamiento
modelo_residual.fit(X_train_visitante, y_train_visitante, epochs=80, batch_size=32, validation_split=0.2, verbose=0)

# Evaluación
loss, mae = modelo_residual.evaluate(X_test_visitante, y_test_visitante, verbose=0)
print(f"🔁 Red residual → MAE: {mae:.4f}")


🔁 Red residual → MAE: 1.0068


In [15]:
from tensorflow.keras.layers import Multiply, Softmax

input_layer = Input(shape=(X_train_visitante.shape[1],))
attention = Dense(X_train_local.shape[1], activation='softmax')(input_layer)
x = Multiply()([input_layer, attention])  # aplica atención

x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
output = Dense(1)(x)

modelo_atencion = Model(inputs=input_layer, outputs=output)
modelo_atencion.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Entrenamiento
modelo_atencion.fit(X_train_visitante, y_train_visitante, epochs=80, batch_size=32, validation_split=0.2, verbose=0)

# Evaluación
loss, mae = modelo_atencion.evaluate(X_test_visitante, y_test_visitante, verbose=0)
print(f"🎯 Red con atención → MAE: {mae:.4f}")


🎯 Red con atención → MAE: 1.0100


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Dropout

modelo_embudo = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_visitante.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1)
])
modelo_embudo.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Entrenamiento
modelo_embudo.fit(X_train_visitante, y_train_visitante, epochs=80, batch_size=32, validation_split=0.2, verbose=0)

# Evaluación
loss, mae = modelo_embudo.evaluate(X_test_visitante, y_test_visitante, verbose=0)
print(f"🏗️ Red tipo embudo → MAE: {mae:.4f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


🏗️ Red tipo embudo → MAE: 1.0315
